In [ ]:
url = 'https://dtmapi.iom.int/api/idpAdmin2Data/GetAdmin2Datav2'

In [ ]:
url

In [ ]:
# 0. Install & configure Java, Spark and helper libs
!apt-get update -qq && apt-get install -y openjdk-8-jdk-headless -qq
!pip install pyspark requests --quiet

In [ ]:
import os, base64
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, sum as spark_sum
import requests

In [ ]:
# 1. Define Medallion directories
base_dir   = "/content/medallion"
bronze_dir = os.path.join(base_dir, "bronze")
silver_dir = os.path.join(base_dir, "silver")
gold_dir   = os.path.join(base_dir, "gold")
for d in (bronze_dir, silver_dir, gold_dir):
    os.makedirs(d, exist_ok=True)

In [ ]:
# 2. Start SparkSession
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
spark = (
    SparkSession.builder
        .appName("DTM_IDP_Medallion")
        .getOrCreate()
)

In [ ]:
# —— BRONZE LAYER: Raw JSON Ingestion with Basic Auth —— #

url = "https://dtmapi.iom.int/api/idpAdmin2Data/GetAdmin2Datav2"

In [ ]:
# Fetch ORG_ID / API_KEY from env vars (or hard-code for testing)
org_id  = os.getenv("DTM_API_ORG_ID", "<YOUR_ORG_ID>")
api_key = os.getenv("DTM_API_KEY", "<YOUR_API_KEY>")

In [ ]:
# Build Basic Auth header
cred_bytes = f"{org_id}:{api_key}".encode()
b64_creds  = base64.b64encode(cred_bytes).decode()
headers    = {"Authorization": f"Basic {b64_creds}"}

In [ ]:
resp = requests.get(url, headers=headers)
resp.raise_for_status()   # will error out (403) if auth fails

In [ ]:
# Save the raw JSON to Bronze
raw_path = os.path.join(bronze_dir, "idp_admin2_raw.json")
with open(raw_path, "w") as f:
    f.write(resp.text)

# Read raw JSON into a Spark DataFrame
bronze_df = spark.read.json(bronze_dir)
print("🔸 Bronze schema:")
bronze_df.printSchema()
print("🔸 Bronze sample rows:")
bronze_df.show(5, truncate=False)

# —— SILVER LAYER: Clean & Flatten —— #
silver_df = bronze_df.select(
    col("admin0Name"),
    col("admin0Pcode"),
    col("admin1Name"),
    col("admin1Pcode"),
    col("admin2Name"),
    col("admin2Pcode"),
    col("numPresentIdpInd"),
    to_date(col("reportingDate"), "yyyy-MM-dd").alias("reporting_date"),
    col("yearReportingDate").alias("year"),
    col("monthReportingDate").alias("month"),
    col("roundNumber").alias("round"),
    col("assessmentType")
)

silver_df.write.mode("overwrite").parquet(silver_dir)
print("🔸 Silver schema:")
silver_df.printSchema()
print("🔸 Silver sample rows:")
silver_df.show(5, truncate=False)

# —— GOLD LAYER: Aggregations & Business Logic —— #
gold_df = (
    spark.read.parquet(silver_dir)
         .groupBy("admin1Name", "reporting_date")
         .agg(spark_sum("numPresentIdpInd").alias("total_idp"))
         .orderBy("admin1Name", "reporting_date")
)

gold_df.write.mode("overwrite").parquet(gold_dir)
print("🔸 Gold schema:")
gold_df.printSchema()
print("🔸 Gold sample rows:")
gold_df.show(5, truncate=False)

# 🎉 Now you have:
#   • BRONZE raw JSON in   /content/medallion/bronze
#   • SILVER parquet data in /content/medallion/silver
#   • GOLD aggregates in    /content/medallion/gold